In [ ]:
pip install tensorflow matplotlib seaborn numpy pandas scikit-learn pillow flask flask-cors pyngrok

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd

# Image augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255, rotation_range=30, zoom_range=0.2, 
    horizontal_flip=True, width_shift_range=0.2, height_shift_range=0.2
)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load dataset
train_data = train_datagen.flow_from_directory(
    "wildlife_dataset/train", target_size=(224, 224), batch_size=64, class_mode='categorical'
)
val_data = val_test_datagen.flow_from_directory(
    "wildlife_dataset/val", target_size=(224, 224), batch_size=64, class_mode='categorical'
)
test_data = val_test_datagen.flow_from_directory(
    "wildlife_dataset/test", target_size=(224, 224), batch_size=64, class_mode='categorical', shuffle=False
)

# Print the number of images in each dataset
print(f"Number of training images: {train_data.samples}")
print(f"Number of validation images: {val_data.samples}")
print(f"Number of test images: {test_data.samples}")

# Automatically detect the number of classes
NUM_CLASSES = len(train_data.class_indices)
class_labels = list(train_data.class_indices.keys())
print(f"Detected {NUM_CLASSES} classes:", train_data.class_indices)

# Load MobileNetV2 with pre-trained ImageNet weights
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze pre-trained layers

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dense(128, activation="relu")(x)
output_layer = Dense(NUM_CLASSES, activation="softmax")(x)

# Create model
model = Model(inputs=base_model.input, outputs=output_layer)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Model summary
model.summary()

# Define callbacks
callbacks = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ModelCheckpoint("best_wildlife_classifier.h5", save_best_only=True, monitor="val_accuracy", mode="max")
]

# Train the model
history = model.fit(train_data, validation_data=val_data, epochs=12, callbacks=callbacks)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Predictions
test_data.reset()
Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_data.classes

# Classification report
report = classification_report(y_true, y_pred, target_names=test_data.class_indices.keys())
print("Classification Report:")
print(report)

# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(12, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

# Save final model
model.save("wildlife_classifier.h5")

# Plot training history
plt.figure(figsize=(8, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training & Validation Accuracy')
plt.show()

: 

In [ ]:
from flask import Flask, render_template, request, jsonify
import tensorflow as tf
import numpy as np
from PIL import Image
import io
import os
from pyngrok import ngrok

app = Flask(__name__)

# Load the trained model
model = tf.keras.models.load_model("wildlife_classifier.h5")

# Get class names from your dataset directory
# You'll need to adjust this path to match your actual dataset location
class_names = sorted(os.listdir("wildlife_dataset/train"))
print(f"Loaded classes: {class_names}")

def preprocess_image(image_bytes):
    """Process uploaded image bytes for model prediction"""
    img = Image.open(io.BytesIO(image_bytes))
    img = img.resize((224, 224))  # Resize to match model's expected input
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    if file:
        # Read and preprocess the image
        img_bytes = file.read()
        img_array = preprocess_image(img_bytes)

        # Make prediction
        predictions = model.predict(img_array)
        predicted_class_index = np.argmax(predictions[0])
        predicted_class = class_names[predicted_class_index]
        confidence = float(predictions[0][predicted_class_index])

        # Get top 3 predictions for display
        top_indices = predictions[0].argsort()[-3:][::-1]
        top_results = [
            {"class": class_names[i], "confidence": float(predictions[0][i])}
            for i in top_indices
        ]

        return jsonify({
            'prediction': predicted_class,
            'confidence': confidence,
            'top_results': top_results
        })

def run_with_ngrok():
    # Set up ngrok
    port = 5000
    public_url = ngrok.connect(port).public_url
    print(f" * Running on {public_url}")
    app.config["BASE_URL"] = public_url
    
    # Start Flask app
    app.run(debug=False, port=port)

if __name__ == '__main__':
    run_with_ngrok()